In [2]:
%matplotlib inline
# not sure if necessary

In [3]:
import finetune
import importlib
import torch

In [4]:
# run this cell when you change stuff in the finetune module, and don't want to restart the notebook's kernel.
importlib.reload(finetune)
finetune.structure_checkpoints()  # If this prints 'Cannot safely create ...', it means that the folder probably already exists

In [ ]:
# inception model doesn't work.
# models = ['resnet', 'alexnet', 'densenet', 'squeezenet', 'vgg']
#
# for name in models:
#     print('[TRAINING]:', name)
# finetune.CLASSIFIER_NAME = name
importlib.reload(finetune)

for classifier_name in ['resnet', 'densenet', 'vgg']:
    for occlusion_name in finetune.OCCLUSION_OPTIONS:
        for dataset in finetune.DATASET_OPTIONS: # ['FGVC-Aircraft' , 'StanfordCars']
            print('STARTING: model={} | dataset={} | occlusion={}'.format(classifier_name, dataset, occlusion_name))
            finetune.CLASSIFIER_NAME = classifier_name
            finetune.NUM_EPOCHS = 20
            finetune.FEATURE_EXTRACT = False # default
            finetune.DATASET = dataset  # [StanfordCars, FGVC-Aircraft]
            finetune.OCCLUSION_NAME = occlusion_name
            finetune.OPTIMIZER_NAME = 'adam' # default
            # finetune.print_hypers() # shows params

            # Initialize the model for this run
            model_ft = finetune.initialize_model(_verbose=False)

            model, hist, state = finetune.finetune_model(model_ft, checkpoint_save=10)

            # Save the model after the last epoch
            finetune.save_model(state)
            del model_ft
            del model
            del hist
            del state


STARTING: model=resnet | dataset=StanfordCars | occlusion=None


Epoch 2/49 train:  63%|██████▎   | 640/1018 [02:13<01:07,  5.64it/s, loss=1.6773, accuracy=0.69111]

In [ ]:
model_save_path = finetune.format_model_path('resnet',
                           'FGVC-Aircraft',
                           30)

checkpoint = finetune.load_checkpoint(model_save_path)

train_acc_history, train_loss_history, val_acc_history = finetune.get_information_from_checkpoint(checkpoint, plot=True)

In [ ]:
optimizer_state_dict = checkpoint['optimizer_state_dict']
finetune.NUM_EPOCHS = 10
checkpoint_model = finetune.get_model_architecture('resnet')
checkpoint_model.load_state_dict(checkpoint['model_state_dict'])

test_model, hist, state = finetune.finetune_model(checkpoint_model,
                                                  'adam',
                                                  optimizer_state_dict=optimizer_state_dict,
                                                  checkpoint_save=5,
                                                  is_retrain=True)


In [ ]:
finetune.save_model(state)